# Echo-A 1B5 (Basemodel Training)
This model has the same settings as raven 1B5 model.

- Layer count: 24
- Embed size: 2048

The goal of this model training is to fully replicate, and match (or exceed) the existing 1B5 telephone model that was finetuned from raven.

This ensures that our training process/replication for this test is valid

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps


# Basic Setup

In [3]:
# First lets get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
#
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!rm -rf ../../../model/Echo-A-1B5-Init.pth
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ../../../model/Echo-A-1B5-Init.pth

--2023-06-22 16:21:18--  https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
Resolving huggingface.co (huggingface.co)... 13.224.249.10, 13.224.249.119, 13.224.249.44, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cb/ef/cbef09abb2634a3375b28868bffa285226dfeabedec89b28c2fb302221164d66/0ec7214ed16737a6348254e6f96d8cdc04d3b5efbd5f53fe9337607ea42b5b9f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Echo-A-1B5-Init.pth%3B+filename%3D%22Echo-A-1B5-Init.pth%22%3B&Expires=1687681279&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2NiL2VmL2NiZWYwOWFiYjI2MzRhMzM3NWIyODg2OGJmZmEyODUyMjZkZmVhYmVkZWM4OWIyOGMyZmIzMDIyMjExNjRkNjYvMGVjNzIxNGVkMTY3MzdhNjM0ODI1NGU2Zjk2ZDhjZGMwNGQzYjVlZmJkNWY1M2ZlOTMzNzYwN2VhNDJiNWI5Zj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb

In [6]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment


## Stage 1 : Foundation model training

In [5]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-A-1B5-enwiki.yaml"

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 89.47it/s]


In [2]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Foundation (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

[2023-07-07 12:25:33,563] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230707_122535-j2ji8e4t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Echo-A-1B5 enwiki_100k foundation (1xGPU, bs=12, ctx=4096)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/j2ji8e4t
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_4096_bf16/build.ninja...
Building extension module wkv_4096_bf16...
Allowing ninja to set a default number of workers... (overridable by se

In [8]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B5-enwiki/last.ckpt"

# Generating rwkv_zero_to_fp32.py file
# Running rwkv_zero_to_fp32.py file, exporting the RWKV model
[2023-07-08 23:22:34,413] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint './checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.gradients, world_size: 1
Parsing checkpoint created by deepspeed==0.9.5
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to rwkv_model.pth
# Exported RWKV model
# RWKV fp32 model is located at ../checkpoint/Echo-A-1B5-enwiki/epoch=0-step=10750.ckpt/rwkv_model.pth


In [10]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B5-enwiki/last.ckpt/rwkv_model.pth ./model/Echo-A-1B5-Stage1.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B5-Stage1.pth

-rw-rw-r-- 1 picocreator picocreator 5.7G Jul  8 23:25 ./model/Echo-A-1B5-Stage1.pth


In [8]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B5-Stage1.pth "cuda fp32"

Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading ../model/Echo-A-1B5-Stage1.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-fl

# Stage 2 : Instruct Tuning

In [35]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-A-1B5-instruct.yaml"

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 889.00it/s]


In [6]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

[2023-07-09 03:31:47,610] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 889882384
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 889882384
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230709_033149-0ypt2hlb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Echo-A-1B5 instruct-tune (1xGPU, bs=12, train-ctx=4096)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/0ypt2hlb
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting 

In [7]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B5-instruct/last.ckpt"

# Generating rwkv_zero_to_fp32.py file
# Running rwkv_zero_to_fp32.py file, exporting the RWKV model
[2023-07-09 12:54:29,556] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint './checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.gradients, world_size: 1
Parsing checkpoint created by deepspeed==0.9.5
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to rwkv_model.pth
# Exported RWKV model
# RWKV fp32 model is located at ../checkpoint/Echo-A-1B5-instruct/last.ckpt/rwkv_model.pth


In [8]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B5-instruct/last.ckpt/rwkv_model.pth ./model/Echo-A-1B5-Stage2.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B5-Stage2.pth

-rw-rw-r-- 1 picocreator picocreator 5.7G Jul  9 12:55 ./model/Echo-A-1B5-Stage2.pth


In [10]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B5-Stage2.pth "cuda fp32"

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ../model/Echo-A-1B5-Stage2.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-float32 17-cuda-float32-float32 18-cuda-float32-float32 19-cuda-float32-float32 20-cuda-float32-float32 21-cuda-float32-float32 22-cuda-float32-float32 23-cuda-float32-float32 24-cuda-float32-float32 
blocks.0.att.key.weight           f32   cuda:0   2048  2048 
blocks.0.att.output.weight        f32   cuda:0   2048  2048 
blocks.0.att.receptance.weight    f32   cuda:0   2048  2048 
blocks.0.att.time_mix_k           f32   cuda:0   2048       


In [7]:
# Lets do a quick memory test
# (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B5-Stage2.pth"

Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /home/picocreator/rwkv-proj/picocreator-memory-experiment/model/Echo-A-1B5-Stage2.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-flo